# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,12.36,93,100,0.89,US,1746363667
1,1,kondopoga,62.2057,34.2614,9.32,38,99,4.43,RU,1746363866
2,2,margaret river,-33.9500,115.0667,17.85,66,99,4.67,AU,1746363868
3,3,slave lake,55.2834,-114.7690,8.99,37,100,6.69,CA,1746363869
4,4,as sulayyil,20.4607,45.5779,39.80,6,100,5.09,SA,1746363870


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    size='Humidity',
    color='Humidity',
    scale=1,
    frame_width=700,
    frame_height=500,
    alpha=0.7,
    hover_cols=["City", "Country", "Humidity"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] >= 20) &
    (city_data_df["Max Temp"] <= 30) &
    (city_data_df["Humidity"] < 50) &
    (city_data_df["Cloudiness"] < 50)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
85,85,hermanus,-34.4187,19.2345,23.22,42,0,2.39,ZA,1746363974
86,86,qurayyat,31.3318,37.3428,28.10,15,40,8.23,SA,1746363975
91,91,bredasdorp,-34.5322,20.0403,23.99,23,0,0.79,ZA,1746363981
120,120,ajdabiya,30.7554,20.2263,26.76,24,0,5.90,LY,1746364017
122,122,sirte,31.2089,16.5887,27.97,27,5,4.75,LY,1746364019


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
85,hermanus,ZA,-34.4187,19.2345,42,
86,qurayyat,SA,31.3318,37.3428,15,
91,bredasdorp,ZA,-34.5322,20.0403,23,
120,ajdabiya,LY,30.7554,20.2263,24,
122,sirte,LY,31.2089,16.5887,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
hermanus - nearest hotel: Aloe guest house
qurayyat - nearest hotel: دانة الشمال
bredasdorp - nearest hotel: Victoria Hotel
ajdabiya - nearest hotel: فندق المدينة
sirte - nearest hotel: فندق المدينة-سرت
hami - nearest hotel: Хами
turpan - nearest hotel: Hua Sheng Hotel
split - nearest hotel: Hotel Central
big bend - nearest hotel: No hotel found
buckeye - nearest hotel: Days Inn
fiambala - nearest hotel: Casona del Pino
kalach - nearest hotel: No hotel found
balykshi - nearest hotel: Арасан
saldanha - nearest hotel: Hoedjesbaai
tokmok - nearest hotel: Жибек
marsa alam - nearest hotel: Blue House Hotel
novouzensk - nearest hotel: No hotel found
la reforma - nearest hotel: No hotel found
vylkove - nearest hotel: Готель Венеція
l'alcora - nearest hotel: No hotel found
kruisfontein - nearest hotel: No hotel found
shakawe - nearest hotel: EdenWood guesthouse
zara - nearest hotel: No hotel found
kashary - nearest hotel: Гостиница


,City,Country,Lat,Lng,Humidity,Hotel Name
85,hermanus,ZA,-34.4187,19.2345,42,Aloe guest house
86,qurayyat,SA,31.3318,37.3428,15,دانة الشمال
91,bredasdorp,ZA,-34.5322,20.0403,23,Victoria Hotel
120,ajdabiya,LY,30.7554,20.2263,24,فندق المدينة
122,sirte,LY,31.2089,16.5887,27,فندق المدينة-سرت


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    size='Humidity',
    color='Humidity',
    scale=1,
    frame_width=700,
    frame_height=500,
    alpha=0.7,
    hover_cols=["City", "Country", "Humidity", "Hotel Name"]
)


# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)